In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/test.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/submit.csv


In [3]:
test = pd.read_csv('/kaggle/input/fake-news/test.csv')
train = pd.read_csv('/kaggle/input/fake-news/train.csv')
sub = pd.read_csv('/kaggle/input/fake-news/submit.csv')

In [4]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
train['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [6]:
train = train.drop(['author', 'id'], axis = 1)

In [7]:
train.dropna(axis = 0,subset=['text'], inplace =True)

In [8]:
train.reset_index(inplace=True)

In [9]:
corpus = train['text']
corpus

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20756    Rapper T. I. unloaded on black celebrities who...
20757    When the Green Bay Packers lost to the Washing...
20758    The Macy’s of today grew from the union of sev...
20759    NATO, Russia To Hold Parallel Exercises In Bal...
20760      David Swanson is an author, activist, journa...
Name: text, Length: 20761, dtype: object

In [10]:
y = train['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20756    0
20757    0
20758    0
20759    1
20760    1
Name: label, Length: 20761, dtype: int64

In [11]:
from keras.utils import to_categorical

In [12]:
y =  to_categorical(y)
y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [15]:
f = open("../input/glove6b50dtxt/glove.6B.50d.txt", encoding='utf8')

In [16]:
embedding_index = {}

for line in f:
    values = line.split()
    word = values[0]
    emb = np.array(values[1:], dtype ='float')
    
    embedding_index[word] = emb

In [17]:
def get_embedding_output(X):
    maxLen = 50
    embedding_output = np.zeros((len(X), maxLen, 50))
    
    for ix in range(X.shape[0]):
#         print(ix)
        my_example = X[ix].split()
        
#         print(my_example)       
        for ij in range(len(my_example)): 
            if (embedding_index.get(my_example[ij].lower()) is not None) and (ij<maxLen):
                embedding_output[ix][ij] = embedding_index[my_example[ij].lower()]
            
    return embedding_output

In [18]:
embeddings = get_embedding_output(corpus)

## SPLIT

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size = 0.2 , random_state=42)

In [21]:
X_train.shape

(16608, 50, 50)

## LSTM

In [22]:
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [23]:
model = Sequential()
model.add(Bidirectional(LSTM(units = 256, return_sequences=True), input_shape = (50,50)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units= 128)))
model.add(Dropout(0.25))

model.add(Dense(units=2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 50, 512)           628736    
_________________________________________________________________
dropout (Dropout)            (None, 50, 512)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               656384    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 1,285,634
Trainable params: 1,285,634
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
hist = model.fit(embeddings, y, validation_split=0.2, shuffle=True, batch_size=32, epochs=20)

Epoch 1/20
145/519 [=======>......................] - ETA: 1:43 - loss: 0.4569 - accuracy: 0.7772

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
test['new_text'] = test['text'].fillna(' ')

In [ ]:
test_data =  get_embedding_output(test['new_text'])

In [ ]:
out = model.predict_classes(test_data)

In [ ]:
out

In [ ]:
data_out = pd.DataFrame({
    'id': test['id'],
    'label': out
})

In [ ]:
data_out

In [ ]:
data_out.to_csv('./submit.csv')